In [1]:
using Statistics
using Printf
using Plots
using LinearAlgebra

using DelimitedFiles

using Base.Threads

In [2]:
function compute_pcf(xs, ys, xi_ar, dxi; Lx::Float64=maximum(xs)-minimum(xs), Ly::Float64=maximum(ys)-minimum(ys))

    N = length(xs)
    nb = length(xi_ar)
    counts_t = zeros(Float64, nb, nthreads())

    r_min = xi_ar[1] - dxi / 2
    r_max = xi_ar[end] + dxi / 2
    r0 = xi_ar[1] - dxi / 2

    @threads for i in 1:N-1
        xi = xs[i]
        yi = ys[i]
        for j in i+1:N
            dx = xs[j] - xi
            dy = ys[j] - yi

            # Apply periodic boundary conditions
            dx -= Lx * round(dx / Lx)
            dy -= Ly * round(dy / Ly)

            r = hypot(dx, dy)
            if (r >= r_min) && (r < r_max)
                k = Int(floor((r - r0) / dxi)) + 1
                if 1 <= k <= nb
                    counts_t[k, threadid()] += 2.0  # *2 for ordered pairs
                end
            end
        end
    end

    counts = sum(counts_t; dims=2)[:]
    pcf = similar(counts)
    area = Lx * Ly
    pair_factor = N * (N - 1)
    area_pref = 2π * dxi / area

    @inbounds for k in 1:nb
        r = xi_ar[k]
        den = pair_factor * (area_pref * r)
        pcf[k] = den > 0 ? counts[k] / den : NaN
    end

    return pcf
end

compute_pcf (generic function with 1 method)

# Horshoe bend PCF

In [ ]:
input_path = "coordinates_samples/horshoe_bend_07_2019_coords_sample.txt"  # your coordinate file

coords = readdlm(input_path)

xs = coords[:, 1]
ys = coords[:, 2]

r_max = 100.0  # maximum radius to consider
dr = 2       # radial bin width

L = 1000.0  # box size

# Radial bins
r_ar = collect(dr/2:dr:r_max)

@printf("Domain L = %.3f, Δr = %.3f, #bins = %d\n", L, dr, length(r_ar))

# Compute PCF
g = compute_pcf(xs, ys, r_ar, dr; Lx=L, Ly=L)

# Save output
# open("horshoe_bend_07_2019_pcf.csv", "w") do io
#     println(io, "r,g")
#     @inbounds for k in 1:length(r_ar)
#         @printf(io, "%.6f,%.6f\n", r_ar[k], g[k])
#     end
# end

Domain L = 1000.000, Δr = 2.000, #bins = 50


In [ ]:
plot(xi_ar, g, xlabel="r", ylabel="g(r)", title="Pair-Correlation Function", legend=false)

# Big system simulation

In [ ]:
input_path = "big_system/P_58.008530460617.txt"  # your coordinate file

coords = readdlm(input_path)

xs = coords[1, :]
ys = coords[2, :]

r_max = 10.0  # maximum radius to consider
dr = 0.1       # radial bin width

@time xi_ar, g = compute_pcf_f(xs, ys, r_max, dr);

# Save output
open("big_system_pcf.csv", "w") do io
    println(io, "r,g")
    @inbounds for k in 1:length(xi_ar)
        @printf(io, "%.6f,%.6f\n", xi_ar[k], g[k])
    end
end
println("Wrote big_system_pcf.csv")

In [ ]:
plot(xi_ar, g, xlabel="r", ylabel="g(r)", title="Pair-Correlation Function", legend=false)